In [1]:
from pathlib import Path

import os
import sys

current_path = Path(os.getcwd())
sys.path.append(str(current_path.parent))

In [2]:
import vectorian
vectorian.compile()

Matching checksum for /Users/arbeit/Projects/vectorian-2021/vectorian/core/cpp/core.cpp --> not compiling


In [3]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [4]:
import vectorian
from vectorian.embeddings import PretrainedFastText, CompressedFastTextVectors, PretrainedGloVe

#fasttext = PretrainedFastText("en")
#fasttext = CompressedFastTextVectors("../models/fasttext-en-mini.kv")
glove = PretrainedGloVe('6B', ndims=50)

In [5]:
import vectorian.utils as utils

token_mappings = {
    "tokenizer": [],
    "tagger": []
}

#token_mappings["tokenizer"].append(utils.lowercase())
token_mappings["tokenizer"].append(utils.erase("W"))
token_mappings["tokenizer"].append(utils.alpha())

def filter_punct(t):
    return None if t["pos"] == "PUNCT" else t

token_mappings["tagger"].append(filter_punct)
token_mappings["tagger"].append(utils.rewrite({
    'pos': {
        'PROPN': 'NOUN'
    },
    'tag': {
        'NNP': 'NN',
        'NNPS': 'NNS',
    }
}))



In [13]:
from vectorian.importers import StringImporter
from vectorian.session import LabSession
from vectorian.metrics import CosineMetric, TokenSimilarityMetric
from vectorian.metrics import AlignmentSentenceMetric
from vectorian.alignment import WatermanSmithBeyer, ConstantGapCost

im = StringImporter(nlp)
doc = im("At the station is an art museum.")

metric = AlignmentSentenceMetric(
    token_metric=TokenSimilarityMetric(glove, CosineMetric()),
    alignment=WatermanSmithBeyer(gap=ConstantGapCost(0), zero=0.25))

session = LabSession(
    [doc],
    static_embeddings=[glove],
    token_mappings=token_mappings)

index = session.partition("sentence").index(metric, nlp)

Importing <string>: 100%|██████████| 1/1 [00:00<00:00, 68.64it/s]


In [14]:
import numpy as np
v = list(session.embeddings)[0].word_vec("railway")
(v / np.linalg.norm(v))[:10]

array([ 0.10564262,  0.09795339,  0.00622643, -0.01882638, -0.26887512,
        0.01853382, -0.182363  ,  0.05094859, -0.03497178, -0.25857762],
      dtype=float32)

In [17]:
def debug(hook, args):
    if hook == 'fill_matrix':
        print(hook, args['t'][:10])
    if hook == 'similarity_matrix':
        print(hook, args)
    #print(hook)

index.find("picture", n=1, debug=debug)

FloatProgress(value=0.0, layout=Layout(width='100%'), max=1.0)

fill_matrix [-0.05316912  0.1222051   0.0792319  -0.03397769  0.21265252 -0.05686188
 -0.11944044 -0.2277743   0.02206284  0.04720843]
fill_matrix [-0.05316912  0.1222051   0.0792319  -0.03397769  0.21265252 -0.05686188
 -0.11944044 -0.2277743   0.02206284  0.04720843]
similarity_matrix {'matrix': array([[0.6026792 ],
       [0.6026792 ],
       [0.3087222 ],
       [0.5977156 ],
       [0.58631784],
       [0.6044975 ],
       [0.4451513 ],
       [1.        ]], dtype=float32), 'rows': <built-in method  of PyCapsule object at 0x7ff9eec71540>, 'columns': <built-in method  of PyCapsule object at 0x7ff9eec71480>}
